In [1]:
# -------------------------

# This notebook includes the analysis tools used in the final presentation

# -------------------------
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import warnings
warnings.simplefilter(action='ignore')
from tqdm import tqdm
from pathlib import Path
import seaborn as sns
import zipfile
import geopandas as gpd
from shapely.geometry import Point, LineString
from scipy import stats
import datetime
import matplotlib.dates as mdates
import plotly.graph_objects as go
from matplotlib.ticker import MaxNLocator
import re
from dateutil.relativedelta import relativedelta
import matplotlib.ticker as mticker


In [2]:
season_queries = {
    "2023-1": "service_date >= '2022-12-18' & service_date <= '2023-03-11'",
    "2023-2": "service_date >= '2023-03-12' & service_date <= '2023-07-02'",
    "2023-3": "service_date > '2023-07-02' & service_date <= '2023-08-26'",
    "2023-4"  : "service_date >= '2023-08-27' & service_date <= '2023-12-16'",
    "2024-1": "service_date >= '2023-12-17' & service_date <= '2024-04-06'",
    "2024-2": "service_date >= '2024-04-07' & service_date <= '2024-06-15'",
    "2024-3": "service_date >= '2024-06-16' & service_date <= '2024-08-24'",
    "2024-4"  : "service_date > '2024-08-24' & service_date <= '2024-12-14'",
    "2025-1": "service_date > '2024-12-14' & service_date <= '2025-04-06'",
    "2025-2": "service_date > '2025-04-06' & service_date <= '2025-06-14'",
}

In [3]:
# -------------------------

# Read pre-calculated dataframes from data_process.ipynb
# df_all: cleaned AVL data
# sd_all: trip-level runtimes
# combined: HASTUS schedules between 2023-1 to 2025-3

# -------------------------


df_all = pd.read_csv('df_all.csv')

df_all["service_date"]     = pd.to_datetime(df_all["service_date"],     format="%Y-%m-%d",              errors="coerce")
df_all["scheduled_boston"] = pd.to_datetime(df_all["scheduled_boston"], format="%Y-%m-%d %H:%M:%S",     errors="coerce")
df_all["actual_boston"]    = pd.to_datetime(df_all["actual_boston"],    format="%Y-%m-%d %H:%M:%S",     errors="coerce")

all_available_routes = df_all.drop_duplicates('route_id')
all_available_routes = all_available_routes[all_available_routes['route_id'].astype(str).str.strip().str.fullmatch(r'\d+')]



sd_all = pd.read_csv('sd_all.csv')
sd_all['service_date']    = pd.to_datetime(sd_all['service_date'])


combined = pd.read_csv('combined.csv')


In [4]:
df_all = df_all.rename(columns={'timepointid':'time_point_id'})


In [5]:
# -------------------------

# One digit routes (such as Route 1) are recorded as '01' in database, so we are assigning a 
# route key to avoid integer/string mismatch

# -------------------------

def _route_key(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()

    m = re.fullmatch(r'0*(\d+)([A-Za-z]*)', s)
    if m:
        num = str(int(m.group(1)))           # drop leading zeros
        suf = m.group(2).upper()             # normalize suffix casing
        return num + suf

    return s.upper()
    
if 'route_key' not in sd_all.columns:
    sd_all = sd_all.copy()
    sd_all['route_key'] = sd_all['route_id'].map(_route_key)

if 'route_key' not in combined.columns:
    combined = combined.copy()
    combined['route_key'] = combined['Route'].map(_route_key)


In [6]:
# -------------------------

# For practicality we are dropping the route names that include a letter (SL1, 34E..)

# -------------------------

combined = combined[combined['Route'].astype(str).str.strip().str.fullmatch(r'\d+')]

# There are some routes that have entries in the schedule but not on in the AVL data after cleaning (714,1013)

combined_28 = combined[combined['Route'].astype(int).isin(sd_all['route_id'].astype(int).unique())]

combined_28['time_dir'] = combined_28['Start'].astype(str)+combined_28['Direction'].astype(str)+ \
         combined_28['rat_id'].astype(str)+combined_28['DOW'].astype(str)+combined_28['type_d'].astype(str)+combined_28['Route'].astype(str)

# There are some routes that have multiple busses running the same schedule (variant?)

cnts = combined_28.value_counts('time_dir').reset_index()
cnts = cnts[cnts['count']==1]
combined_28 = combined_28[combined_28['time_dir'].isin(cnts['time_dir'])]

In [7]:
df_all['delay_min'] = (df_all['actual_boston'] - df_all['scheduled_boston'])\
                        .dt.total_seconds() / 60

starts = df_all[(df_all['point_type']=='Startpoint')]
id_keeps = starts[(starts['delay_min']<=15)&(starts['delay_min']>-10)]['half_trip_id']

df_all_cleaned = df_all[df_all['half_trip_id'].isin(id_keeps)]

sd_all_cleaned = sd_all[sd_all['half_trip_id'].isin(id_keeps)]


In [8]:
print(len(df_all_cleaned)/len(df_all))
print(len(sd_all_cleaned)/len(sd_all))

0.9589006370383601
0.9605531021196033


In [9]:

sd_all_mrg = sd_all.merge(combined_28[['time_dir','Act_Layover','Variant']])


sd_all_mrg = (
    sd_all_mrg[pd.to_numeric(sd_all_mrg['Act_Layover'], errors='coerce').notna()]
      .assign(Act_Layover=lambda df: df['Act_Layover'].astype(float))
)

sd_all_mrg['schdLO']  = sd_all_mrg['scheduled_run'] + sd_all_mrg['Act_Layover']


agg = sd_all_mrg.groupby('time_dir').agg(
    actual_run_q90=('actual_run', lambda x: x.quantile(0.90)),
    mean_schd     =('scheduled_run',    'mean'),
    mean_lo     =('Act_Layover',    'mean')).reset_index()

df_out = sd_all_mrg.merge(
agg[['time_dir','actual_run_q90','mean_schd','mean_lo']],
on='time_dir',
how='left'
)
df_out = df_out.drop_duplicates('time_dir')[['actual_run_q90','mean_schd','mean_lo','time_dir','season','holiday','direction_id','schedule_type','hour_Startpoint',
                                             'scheduledhm_Startpoint','DOW','route_id','schdLO','Variant']]


In [10]:
def deficiency_ratio(df: pd.DataFrame, bin_col: str) -> pd.Series:
    
        g = (df
             .groupby(['route_key', bin_col], as_index=False)
             .agg(actual_run=('actual_run', lambda x: x.quantile(0.9)),  
                  schdLO     =('schdLO',     'mean')))
    
        g['pos_diff'] = (g['actual_run'] - g['schdLO']).clip(lower=0)
    
        sums = g.groupby('route_key').agg(
            num=('pos_diff', 'sum'),
            den=('schdLO',   'sum')
        )
        return sums['num'] / sums['den']

In [11]:
def surplus_ratio(df: pd.DataFrame, bin_col: str) -> pd.Series:
    
        g = (df
             .groupby(['route_key', bin_col], as_index=False)
             .agg(actual_run=('actual_run', lambda x: x.quantile(0.9)),   
                  schdLO     =('schdLO',     'mean')))
    
        g['pos_diff'] = (g['schdLO'] - g['actual_run']).clip(lower=0)
    
        sums = g.groupby('route_key').agg(
            num=('pos_diff', 'sum'),
            den=('schdLO',   'sum')
        )
        return sums['num'] / sums['den']

In [12]:
outs = pd.DataFrame()
outs2 = pd.DataFrame()

for season in tqdm(['2025-1','2024-1','2025-2','2024-2','2024-3','2023-3','2024-4','2023-4']):
    for dir in ['Outbound','Inbound']:
        mask = (
            (sd_all_mrg['season'] == season) &
            (sd_all_mrg['direction_id'] == dir) &
            (sd_all_mrg['DOW'] == 'Weekday')
        )
        core = sd_all_mrg.loc[mask,  
                              ['route_key', 'hour_Startpoint', 'scheduledhm_Startpoint',
                               'actual_run', 'schdLO']]
        
        defs_rt = deficiency_ratio(core, 'scheduledhm_Startpoint') 
        surp_rt = surplus_ratio(core, 'scheduledhm_Startpoint')  

        
        out = (pd.concat({'Deficit Percent': defs_rt}, axis=1)
                 .reset_index()
                 .rename(columns={'route_key': 'route'}))

        out2 = (pd.concat({'Surplus Percent': surp_rt}, axis=1)
                 .reset_index()
                 .rename(columns={'route_key': 'route'}))
        
        out['season'] = season
        out['dir'] = dir

        out2['season'] = season
        out2['dir'] = dir
        
        outs = pd.concat([outs,out])
        outs2 = pd.concat([outs2,out2])


100%|██████████| 8/8 [01:43<00:00, 12.91s/it]


In [13]:
outs_ = outs.groupby(['route','season']).mean('Deficit Percent').reset_index()
outs_2 = outs2.groupby(['route','season']).mean('Surplus Percent').reset_index()

In [17]:
#defs = pd.read_excel('Schedule Deficiency one route_data (2).xlsx')

def build_rating_comparison_excel(defs: pd.DataFrame, out_path="route_rating_deltas.xlsx"):
    
    val_col = "Deficit Percent"
    df = (defs[['route', 'season', val_col]]
          .copy())
    df['route'] = df['route'].astype(str).str.strip()
    df = df.groupby(['route', 'season'], as_index=False)[val_col].mean()

    pair_map = {
        '2025-2': '2024-2',
        '2025-1': '2024-1',
        '2024-3': '2023-3',
        '2024-4'  : '2023-4',
    }

    rows = []
    for cur, prev in pair_map.items():
        cur_df  = df[df['season'] == cur][['route', val_col]].rename(columns={val_col: 'current_value'})
        prev_df = df[df['season'] == prev][['route', val_col]].rename(columns={val_col: 'prior_value'})
        merged = pd.merge(cur_df, prev_df, on='route', how='inner')
        if merged.empty:
            continue
        merged['pair'] = f'{cur} vs {prev}'
        merged['current_rating'] = cur
        merged['prior_rating'] = prev
        merged['change'] = merged['current_value'] - merged['prior_value']
        rows.append(merged[['route','pair','prior_rating','prior_value','current_rating','current_value','change']])

    if not rows:
        raise ValueError("No overlapping season pairs found for the provided data.")

    long = pd.concat(rows, ignore_index=True)
    long = long.sort_values(['route','pair'])

    matrix = (long
              .pivot(index='route', columns='pair', values='change')
              .sort_index())
    
    THRESH = 0.005
    red_count = (matrix > THRESH).sum(axis=1)           # how many red cells per route
    pos_sum   = matrix.clip(lower=0).sum(axis=1)        # sum of positive changes
    order = (pd.DataFrame({'red': red_count, 'pos_sum': pos_sum}, index=matrix.index)
             .sort_values(['red', 'pos_sum'], ascending=[False, False]).index)
    matrix = matrix.loc[order]
    with pd.ExcelWriter(out_path, engine='xlsxwriter') as writer:
        long.to_excel(writer, sheet_name='Long', index=False)
        matrix.to_excel(writer, sheet_name='Matrix')

        wb = writer.book

        pct_fmt   = wb.add_format({'num_format': '0.0%'})
        red_fmt   = wb.add_format({'bg_color': '#FFC7CE', 'font_color': '#000000', 'num_format': '0.0%'})
        green_fmt = wb.add_format({'bg_color': '#C6EFCE', 'font_color': '#000000', 'num_format': '0.0%'})
        yellow_fmt= wb.add_format({'bg_color': '#FFEB9C', 'font_color': '#000000', 'num_format': '0.0%'})

        ws = writer.sheets['Matrix']
        ws.set_column(0, 0, 12) 
        if not matrix.empty:
            nrows, ncols = matrix.shape
            first_row, first_col = 1, 1 
            last_row  = first_row + nrows - 1
            last_col  = first_col + ncols - 1

            ws.set_column(first_col, last_col, 14, pct_fmt)

            ws.conditional_format(first_row, first_col, last_row, last_col,
                                  {'type': 'cell', 'criteria': '>', 'value': 0.005, 'format': red_fmt})
            ws.conditional_format(first_row, first_col, last_row, last_col,
                                  {'type': 'cell', 'criteria': '<', 'value': -0.005, 'format': green_fmt})
            ws.conditional_format(first_row, first_col, last_row, last_col,
                                  {'type': 'cell', 'criteria': 'between',
                                   'minimum': -0.005, 'maximum': 0.005, 'format': yellow_fmt})
            ws.freeze_panes(1, 1)

        ws2 = writer.sheets['Long']
        ws2.set_column(0, len(long.columns)-1, 14)
        for col_name in ['prior_value', 'current_value', 'change']:
            cidx = list(long.columns).index(col_name)
            ws2.set_column(cidx, cidx, 12, pct_fmt)

        cidx = list(long.columns).index('change')
        if len(long) > 0:
            ws2.conditional_format(1, cidx, len(long), cidx,
                                   {'type': 'cell', 'criteria': '>', 'value': 0.05, 'format': red_fmt})
            ws2.conditional_format(1, cidx, len(long), cidx,
                                   {'type': 'cell', 'criteria': '<', 'value': -0.05, 'format': green_fmt})
            ws2.conditional_format(1, cidx, len(long), cidx,
                                   {'type': 'cell', 'criteria': 'between',
                                    'minimum': -0.05, 'maximum': 0.05, 'format': yellow_fmt})
            ws2.freeze_panes(1, 0)

    return out_path, long, matrix

out_path, long_df, matrix_df = build_rating_comparison_excel(outs_, "route_rating_deltas.xlsx")
print("Saved:", out_path)


Saved: route_rating_deltas.xlsx
